# Перевод статьи [*Simpson characters detection - Keras CNN - Acc 97%*](https://www.dsimb.inserm.fr/~ghouzam/personal_projects/Simpson_character_recognition.html)

**Оглавление**
* [Введение](#Введение)<br>
* [Подготовка данных](#Подготовка-данных)<br>
    * [Загрузка и подготовка данных](#Загрузка-и-подготовка-данных)<br>
    * [Нормализация](#Нормализация)<br>
    * [Кодирование меток](#Кодирование-меток)<br>
* [Сверточная Нейронная Сеть](#Сверточная-Нейронная-Сеть)<br>
    * [Определение модели](#Time-Features)<br>
    * [Определение оптимизатора и отжига](#Определение-оптимизатора-и-отжига)<br>
    * [Увеличение и подгонка данных](#Увеличение-и-подгонка-данных)<br>
* [Оценка модели](#Оценка-модели)<br>
    * [Матрица неточностей](#Матрица-неточностей)<br> 
    * [Некоторые примеры](#Некоторые-примеры)<br> 

# Введение
Я фанат Симпсонов и смотрю много серий Симпсонов. Персонажи Симпсонов настолько просты и нетрадиционны, что делает их невероятными. Я хотел построить нейронную сеть, которая может распознавать персонажей Симпсонов. Построенная модель очень точна (97%) и может быть использована для установления статистики встречаемости в сериях Симпсонов.

В общем, это мой личный проект. Цель состоит в том, чтобы определять персонажей Симпсонов индивидуально по фотографиям, а затем по видео с помощью глубокого обучения.

Я использовал сверточную нейронную сеть, обученную только на 20 тысячах изображений (около 1 тысячи изображений на персонажа). Модель выучила 20 персонажей Симпсонов. Популярные из них (Гомер, Барт, Лиза ...) были изучены с более чем 1k изображений, в то время как нечастые персонажи были изучены только с 400 изображениями.

Я решил построить модель с помощью Keras API (TensorFlow backend), который очень интуитивно понятен. Во-первых, я подготовлю данные (изображения персонажей симпсонов), а затем сосредоточусь на моделировании и оценке СНС. Наконец, я сделаю быструю демонстрацию прогностической модели с видео.

Я достиг 97,77% точности с помощью этого СНС, обученного за 15 часов на одном процессоре (I5 2500k). С >= 3.0 возможности графического процессора (видеокарты GTX 650) можно использовать tensorflow-gpu с keras. Вычисление будет намного быстрее!

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.font_manager import FontProperties
import scipy
import imageio
import PIL
from PIL import Image
import cv2
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import model_from_json

import warnings

warnings.filterwarnings('ignore')

# Подготовка данных
## Загрузка и подготовка данных
Для этого проекта я использовал данные персонажей Симпсонов, доступные на kaggle:
[The Simpsons Characters Data](https://www.kaggle.com/alexattia/the-simpsons-characters-dataset)

Этот набор данных состоит из 20 папок для процесса обучения (по одной для каждого персонажа), в каждой папке 400-2000 картинок. Набор также включает в себя папку трэйна, где 990 помеченных изображений.

Поэтому сначала я определяю две функции load_train_set и load_test_set, которые загружают, изменяют размер и готовят изображения из набора трэйна и тестового набора соответственно.

Размер каждого изображения изменяется до формата - 64 на 64 пикселя.

In [ ]:
dict_characters = {0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson', 
        3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel', 
        7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lenny_leonard', 11:'lisa_simpson',
        12: 'marge_simpson', 13: 'mayor_quimby',14:'milhouse_van_houten', 15: 'moe_szyslak', 
        16: 'ned_flanders', 17: 'nelson_muntz', 18: 'principal_skinner', 19: 'sideshow_bob'}

In [ ]:
# Load the data
def load_train_set(dirname,dict_characters):
    X_train = []
    Y_train = []
    for label,character in dict_characters.items():
        list_images = os.listdir(dirname+'/'+character)
        for image_name in list_images:
            image = imageio.imread(dirname+'/'+character+'/'+image_name)
            X_train.append(np.array(Image.fromarray(image).resize(size=(64, 64))))
            Y_train.append(label)
    return np.array(X_train), np.array(Y_train)

X_train, Y_train = load_train_set("../input/the-simpsons-characters-dataset/simpsons_dataset", dict_characters)

In [ ]:
# load the test data
def load_test_set(dirname,dict_characters):
    X_test = []
    Y_test = []
    for image_name in os.listdir(dirname):
        character_name = "_".join(image_name.split('_')[:-1])
        label = [label for label,character in dict_characters.items() if character == character_name][0]
        image = imageio.imread(dirname+'/'+image_name)
        X_test.append(np.array(Image.fromarray(image).resize(size=(64, 64))))
        Y_test.append(label)
    return np.array(X_test), np.array(Y_test)

X_test, Y_test = load_test_set("../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset", dict_characters)

## Нормализация

In [ ]:
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

Как только данные загрузятся, я выполняю нормализацию оттенков серого, чтобы уменьшить эффект различий в освещенности.

Более того, СНС сходится быстрее на данных [0..1], чем на данных [0..255].

Я просто делю значения RGB на 255, что является максимальным значением RGB.

Давайте посмотрим образцы. Но прежде я определю две полезные функции, одну, которая случайным образом отображает 20 изображений с их фактическими метками и предсказанными (если все аргументы приведены). Другая функция случайным образом выбирает один элемент для каждого типа в массиве. Эта последняя функция будет использоваться для выбора одного из персонажей Симпсонов для отображения.

In [ ]:
def display_samples(samples_index,imgs,obs, preds_classes=None,preds=None):
    """Эта функция случайным образом отображает 20 изображений с их фактическими метками 
    и предсказуемыми метками (если предусмотрены preds_classes и preds)"""
    n = 0
    nrows = 4
    ncols = 5
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(12,10))
    plt.subplots_adjust(wspace=0, hspace=0)
    for row in range(nrows):
        for col in range(ncols):
            index = samples_index[n]
            ax[row,col].imshow(imgs[index])
            
            actual_label = dict_characters[obs[index]].split("_")[0]
            actual_text = "Actual : {}".format(actual_label)
            
            ax[row,col].add_patch(patches.Rectangle((0, 53),64,25,color='white'))
            font0 = FontProperties()
            font = font0.copy()
            font.set_family("fantasy")
            
            ax[row,col].text(1, 54, actual_text , horizontalalignment='left', fontproperties=font,
                    verticalalignment='top',fontsize=10, color='black',fontweight='bold')
            
            if np.array(preds_classes).size > 1 and np.array(preds).size > 1:
                predicted_label = dict_characters[preds_classes[index]].split('_')[0]
                predicted_proba = max(preds[index])*100
                predicted_text = "{} : {:.0f}%".format(predicted_label,predicted_proba)
            
                ax[row,col].text(1, 59, predicted_text , horizontalalignment='left', fontproperties=font,
                    verticalalignment='top',fontsize=10, color='black',fontweight='bold')
            n += 1

In [ ]:
def pick_up_random_element(elem_type,array):
    """Эта функция случайным образом выбирает один элемент для каждого типа в массиве"""
    return int(random.choice(np.argwhere(array == elem_type)))

In [ ]:
samples = [pick_up_random_element(elem_type,Y_train) for elem_type in range(20)]

display_samples(samples,X_train,Y_train)

![](...%0AB+53BQfLWU3vAAAAAElFTkSuQmCC)

Таким образом, вы можете видеть, что фотографии взяты непосредственно из эпизодов "Симпсонов". Поэтому модели приходится иметь дело с замаскированными или частично скрытыми персонажами. Это сделает нашу модель надежной для обобщения.

## Кодирование меток

In [ ]:
# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = 20)
Y_test = to_categorical(Y_test, num_classes = 20)

Метки - это 20 номеров персонажей от 0 до 19. Нам нужно закодировать эти метки в один горячий вектор (например: "bart_simpson" -> 2 -> [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).

# Сверточная Нейронная Сеть
## Определение модели
Я использовал последовательный API Keras, где вам нужно просто добавлять один слой за раз, начиная с входных данных.

Первый - это сверточный (Conv2D) слой. Это как набор обучаемых фильтров. Я решил установить 32 фильтра для двух первых слоев conv2D и далее 64 и 86 фильтров для двух последних. Каждый фильтр преобразует часть изображения (определяемую размером ядра) с помощью фильтра ядра. Матрица фильтра ядра применяется ко всему изображению. Фильтры можно рассматривать как преобразование изображения.

СНС может изолировать особенности, которые полезны везде, от этих преобразованных изображений (карт особенностей).

Второй важный слой в СНС - это слой пула (MaxPool2D). Этот слой просто действует как фильтр понижающей дискретизации. Он смотрит на N соседних пикселей и выбирает максимальное значение. Они используются для снижения вычислительных затрат и в какой-то степени также уменьшают переобучение. Мы должны выбрать размер пула (то есть размер области, объединяемой каждый раз), чем выше размер пула, тем важнее понижающая дискретизация.

Комбинируя сверточные и объединяющие слои, СНС могут комбинировать локальные особенности и изучать более глобальные особенности изображения.

Отсев - это метод регуляризации, при котором доля узлов в слое случайным образом игнорируется (устанавливая их значения равными нулю) для каждой обучающей выборки. Это случайно отбрасывает предложение сети и заставляет сеть изучать особенности распределенным способом. Эта техника также улучшает обобщение и уменьшает переобучение.

"relu" - это выпрямитель (функция активации max (0, x)). Функция активации выпрямителя используется для добавления нелинейности в сеть.

Пакетная нормализация - это технический трюк, позволяющий ускорить обучение. Например, как мы сделали с шагом нормализации в качестве шага предварительной обработки, чтобы сделать данные сопоставимыми между особенностями, пакетная нормализация нормализует веса и параметры, чтобы данные снова не были слишком большими или слишком маленькими ("внутренний ковариационный сдвиг"). Нормализация данных в каждом мини-пакете позволяет в значительной степени избежать этой проблемы.

Слой Flatten используется для преобразования карт особенностей в один единственный 1D-вектор. Этот шаг выравнивания необходим для того, чтобы вы могли использовать полностью связанные слои после некоторых сверточных/maxpool слоев. Он объединяет в себе все найденные локальные особенности предыдущих сверточных слоев.

В конце я использовал особенности в двух полностью связанных (Dense) слоях, которые являются просто искусственным классификатором нейронных сетей (ANN). В последнем слое (Dense(20,activation="softmax")) сеть выводит распределение вероятностей каждого класса.

In [ ]:
# Установка модели СНС
# Моя архитектура СНС -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*3 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (64,64,3)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 86, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 86, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation = "softmax"))

## Определение оптимизатора и отжига
Как только наши слои добавлены в модель, нам нужно настроить функцию оценки, функцию потерь и алгоритм оптимизации.

Наиболее важной функцией является оптимизатор. Это функция, которая будет итеративно улучшать параметры (фильтровать значения ядра, веса и смещения нейронов ...) и минимизировать потери.

Я выбрал RMSprop (со значениями по умолчанию), это очень эффективный оптимизатор. Модификация RMSProp настраивает метод Adagrad очень простым способом, пытаясь уменьшить его агрессивную, монотонно снижающуюся скорость обучения. Мы также могли бы использовать оптимизатор Стохастического Градиентного Спуска (SGD), но он медленнее, чем RMSprop.

Мы определяем функцию потерь, чтобы измерить, насколько плохо наша модель работает на изображениях с известными метками. Это частота ошибок между фактическими метками и предсказанными. Мы используем специфическую форму для категориальных классификаций (>2 классов), называемую "categorical_crossentropy".

Метрическая функция "accuracy" используется именно для оценки производительности нашей модели. Эта метрическая функция аналогична функции потерь, за исключением того, что результаты оценки метрики не используются при обучении модели (только для оценки).

In [ ]:
# Определение оптимизатора
optimizer = RMSprop(lr=0.001, decay=1e-6)

Поскольку я уже обучил модель в течение 15 часов, оптимальные веса уже были вычислены, и я буду использовать их для загрузки модели.

In [ ]:
PRETRAINED = False

if PRETRAINED :
    json_file = open('best_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
   
    model = model_from_json(loaded_model_json)
    # загрузка весов в модель
    model.load_weights("best_model.hdf5")

In [ ]:
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Установка learning_rate отжига
learning_rate_reduction = ReduceLROnPlateau(monitor="val_accuracy", 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

Чтобы заставить оптимизатор сходиться быстрее и ближе всего к глобальному минимуму функции потерь, я решил использовать метод отжига скорости обучения (LR).

LR - это шаг, с помощью которого оптимизатор проходит через "loss landscape". Чем выше LR, тем больше шаги и тем быстрее сходимость. Однако выборка очень плохая с высоким LR, и оптимизатор, вероятно, может упасть в локальный минимум.

Лучше иметь снижающуюся скорость обучения во время обучения, чтобы эффективно достичь глобального минимума функции потерь.

Чтобы сохранить преимущество быстрого времени вычислений с высоким LR, я уменьшал LR динамически каждые X шагов (эпох) в зависимости от необходимости (точность не улучшается).

С помощью функции ReduceLROnPlateau от Keras.callbacks я решаю уменьшить LR наполовину, если точность не улучшится через 3 эпохи.

In [ ]:
save_best = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

## Увеличение и подгонка данных
Чтобы избежать проблемы переобучения, нам нужно искусственно расширить наш набор данных. Вы можете сделать существующий набор данных еще больше. Идея состоит в том, чтобы изменить обучающие данные с помощью небольших преобразований, чтобы воспроизвести вариации, происходящие, например, во время фильма Симпсона.

Например, когда персонаж не центрирован, масштаб не тот же самый (некоторые пишут большими / маленькими числами), изображение поворачивается...

Подходы, которые изменяют обучающие данные таким образом, чтобы изменить представление массива, сохраняя при этом метку одинаковой, известны как методы увеличения данных (data augmentation techniques). Некоторые популярные дополнения, которые люди используют, - это оттенки серого, горизонтальные перевороты, вертикальные перевороты, случайное кадрирование, дрожание цвета, переводы, повороты и многое другое.

Применив всего пару таких преобразований к нашим обучающим данным, Вы можете легко удвоить или утроить количество обучающих примеров и создать очень надежную модель.

Разница очень важна :
* без увеличения данных я получил точность 93,5%
* с увеличением данных я достиг 97,77% точности

В этой статье этот шаг уже был сделан раньше, поэтому он автоматически обходится.

In [ ]:
if PRETRAINED == False:
    
    epochs = 2 #60 
    batch_size = 32
    
    
    # Увеличение данных для предотвращения переобучения (точность 0,97)
    
    datagen = ImageDataGenerator(
        featurewise_center=False,  # установка входного среднего значения равного 0 для набора данных
        samplewise_center=False,  # установка для каждой выборки среднего значения равного 0
        featurewise_std_normalization=False,  # разделение входных данных на std набора данных
        samplewise_std_normalization=False,  # разделение каждого входа на его std
        zca_whitening=False,  # применения ZCA отбеливания
        rotation_range=10,  # произвольное вращение изображения в диапазоне (Градусы от 0 до 180)
        zoom_range = 0.1, # случайное масштабирование изображения
        width_shift_range=0.1,  # произвольное смещение изображений по горизонтали (доля от общей ширины)
        height_shift_range=0.1,  # произвольное смещение изображений по вертикали (доля от общей высоты)
        horizontal_flip=True,  # случайный переворот изображения по горизонтали
        vertical_flip=False)  # случайный переворот изображения по вертикали
    
    
    datagen.fit(X_train)
    
    # Помещение в модель
    history = model.fit(datagen.flow(X_train,Y_train, batch_size=batch_size, shuffle=True),
                                  epochs = epochs, validation_data = (X_test,Y_test),
                                  steps_per_epoch=X_train.shape[0] // batch_size, 
                                  callbacks=[learning_rate_reduction, save_best])
    
    
    fig, ax = plt.subplots(2,1)
    ax[0].plot(history.history['loss'], color='b', label="Training loss")
    ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
    legend = ax[0].legend(loc='best', shadow=True)
    
    ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
    ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
    legend = ax[1].legend(loc='best', shadow=True)
    
    
    # сериализация модели в JSON
    model_json = model.to_json()
    with open("best_model.json", "w") as json_file:
        json_file.write(model_json)
        # сериализация весов в HDF5
        model.save_weights("best_model.hdf5")

Для увеличения данных я выбираю :

* Произвольный поворот некоторых обучающих изображений на 10 градусов
* Случайное увеличение на 10% некоторых обучающих изображений
* Произвольный сдвиг изображения по горизонтали на 10% ширины
* Произвольный сдвиг изображения по вертикали на 10% высоты
* Случайный переворот изображения по горизонтали

Как только наша модель будет готова, мы поместим обучающий набор данных.

# Оценка модели
## Матрица неточностей

In [ ]:
# прогнозирование результатов
loss, acc = model.evaluate(X_test,Y_test,verbose = 0)

print("Simpson characters were predicted with a loss of {:.5f} and an accuracy of {:.2f}%".format(loss,acc*100))

Персонажи Симпсонов были предсказаны с потерей 0,09837 и точностью 97,78%.

Модель достигает почти 98% (97,7+%) точности на валидационном наборе данных после 60 эпох.

Наша модель очень хорошо обучена !!!

Давайте посмотрим на матрицу неточностей.

Матрица неточностей может быть очень полезна, чтобы увидеть недостатки вашей модели. Я строю матрицу неточностей результатов проверки.

In [ ]:
# Просмотр матрицы неточностей

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Эта функция печатает и строит матрицу неточностей.
    Нормализация может применяться при установке `normalize=True`.
    """
    plt.figure(figsize = (10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Прогнозирование значений из проверочного набора данных
Y_pred = model.predict(X_test)
# Преобразование горячих векторов результатов прогнозирования в список классов
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Преобразование горячих векторов проверочных наблюдений в список классов
Y_true = np.argmax(Y_test,axis = 1) 
# Вычисление матрицы неточностей
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# Построение матрицы неточностей
plot_confusion_matrix(confusion_mtx, classes = list(dict_characters.values())) 

![](...%0ARPTe/wLbO1I5oKAiTAAAAABJRU5ErkJggg==)

Здесь мы видим, что наша СНС очень хорошо работает со всеми персонажами Симпсонов, с небольшим количеством ошибок, учитывая размер тестового набора (990 изображений).

Однако, похоже, у нашей СНС есть некоторые небольшие проблемы с распознаванием Гомера Симпсона, Абрахама Симпсона, Лизы и Барта Симпсона. 3 Барта классифицируются как Лиза Симпсон и 3 Абрахама классифицируются как Гомер Симпсон. Для Лизы и Барта моя гипотеза заключается в том, что у них обоих колючие стрижки, которые вводят модель в заблуждение. Для Абрахама и Гомера - "like father, like son!".

Тем не менее эти ошибки очень редкие, модель невероятно точна.

Давайте посмотрим несколько примеров.

## Некоторые примеры

In [ ]:
# Отображение некоторых результатов
samples = [pick_up_random_element(elem_type,Y_true) for elem_type in range(20)]
display_samples(samples, X_test, Y_true, Y_pred_classes, Y_pred)

![](...)